In [2]:
import json
import os
import re
import glob

import pandas as pd

In [4]:
rows = []
for d in sorted(glob.glob('../logdirs/20190315-sketch/*')):
    exp_name = os.path.basename(d)
    upd_steps, max_steps, batch_size = re.match(
        'upd_steps=(\d+),max_steps=(\d+),batch_size=(\d+)', exp_name).groups()

    eval_paths = glob.glob(os.path.join(d, 'eval-val-step*-bs1.jsonl'))
    all_scores = []
    best = (0, 0)
    for eval_path in eval_paths:
        step = int(re.search('step(\d+)', eval_path).group(1))
        scores = json.load(open(eval_path))
        #best = max(best, (scores['total_scores']['all']['exact'], step))
        all_scores.append((scores['total_scores']['all']['exact'], step))
    all_scores.sort(reverse=True)
    if len(all_scores) == 0:
        all_scores = [(0, 0)]
    if len(all_scores) < 2:
        all_scores += all_scores
    rows.append((upd_steps, max_steps, batch_size, len(eval_paths),
                 all_scores[0][1], all_scores[0][0], all_scores[1][1], all_scores[1][0]))
    print(d)
df = pd.DataFrame(rows, columns=('upd_steps', 'max_steps', 'batch_size', 'num steps eval', 'step', 'acc', 'step', 'acc'))

../logdirs/20190315-sketch/upd_steps=0,max_steps=40000,batch_size=10
../logdirs/20190315-sketch/upd_steps=0,max_steps=40000,batch_size=20
../logdirs/20190315-sketch/upd_steps=0,max_steps=80000,batch_size=10
../logdirs/20190315-sketch/upd_steps=0,max_steps=80000,batch_size=20
../logdirs/20190315-sketch/upd_steps=4,max_steps=40000,batch_size=10
../logdirs/20190315-sketch/upd_steps=4,max_steps=40000,batch_size=20
../logdirs/20190315-sketch/upd_steps=4,max_steps=80000,batch_size=10
../logdirs/20190315-sketch/upd_steps=4,max_steps=80000,batch_size=20


In [6]:
df

,upd_steps,max_steps,batch_size,num steps eval,step,acc,step,acc
0,0,40000,10,20,38100,0.624758,32100,0.621857
1,0,40000,20,20,10100,0.642166,38100,0.636364
2,0,80000,10,38,8100,0.627660,10100,0.618956
3,0,80000,20,21,10100,0.641199,12100,0.628627
4,4,40000,10,20,40000,0.630561,34100,0.630561
5,4,40000,20,2,4100,0.542553,2100,0.488395
6,4,80000,10,36,70100,0.648936,68100,0.647969
7,4,80000,20,24,32100,0.624758,28100,0.623791


In [7]:
best_stats = json.load(
    open('../logdirs/20190315-sketch/upd_steps=4,max_steps=80000,batch_size=10/eval-val-step68100-bs1.jsonl'))

In [8]:
best_stats['total_scores']['all']

{'count': 1034,
 'exact': 0.6479690522243714,
 'exec': 0,
 'partial': {'IUEN': {'acc': 0.4875,
   'acc_count': 80,
   'f1': 0.49367088607594933,
   'rec': 0.5,
   'rec_count': 78},
  'and/or': {'acc': 0.9605662285136501,
   'acc_count': 989,
   'f1': 0.9557344064386318,
   'rec': 0.950950950950951,
   'rec_count': 999},
  'group': {'acc': 0.8423076923076923,
   'acc_count': 260,
   'f1': 0.8279773156899811,
   'rec': 0.8141263940520446,
   'rec_count': 269},
  'group(no Having)': {'acc': 0.8730769230769231,
   'acc_count': 260,
   'f1': 0.8582230623818524,
   'rec': 0.8438661710037175,
   'rec_count': 269},
  'keywords': {'acc': 0.7377232142857143,
   'acc_count': 896,
   'f1': 0.7502837684449489,
   'rec': 0.7632794457274826,
   'rec_count': 866},
  'order': {'acc': 0.7533632286995515,
   'acc_count': 223,
   'f1': 0.7304347826086957,
   'rec': 0.7088607594936709,
   'rec_count': 237},
  'select': {'acc': 0.9148936170212766,
   'acc_count': 1034,
   'f1': 0.9148936170212766,
   'rec':

In [11]:
infer_stats = [json.loads(
    l) for l in open('../logdirs/20190315-sketch/upd_steps=4,max_steps=80000,batch_size=10/infer-val-step68100-bs1.jsonl')]

In [13]:
infer_stats[1]['beams'][0]['inferred_code']

'SELECT Count(some_col) FROM stadium'

In [29]:
for i in range(6, 1000):
    if best_stats['per_item'][i]['partial']['where']['acc'] == 0:
        print(i)

6
18
19
24
25
29
30
45
48
51
52
55
61
62
63
64
65
66
68
69
75
76
79
80
83
84
86
88
96
101
102
104
115
117
118
119
124
129
130
131
132
135
147
148
151
152
153
154
157
158
159
160
161
166
167
168
170
171
172
174
177
178
179
180
182
189
197
215
217
243
244
257
258
265
266
269
280
289
290
293
294
327
346
363
377
378
380
391
392
410
413
417
418
420
422
423
424
427
435
436
440
457
458
465
467
485
501
504
507
512
521
533
534
535
536
544
554
559
560
563
567
568
569
584
592
595
598
601
602
609
611
613
615
616
617
618
619
621
623
625
626
629
630
633
634
635
636
637
638
641
642
643
644
645
651
652
662
684
695
697
712
719
722
723
724
725
726
732
738
739
744
745
748
756
757
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
784
785
786
787
792
795
796
798
799
808
809
818
820
830
831
841
846
847
860
897
898
899
900
901
910
914
915
918
919
921
924
925
926
927
928
929
932
936
937
944
954
955
959
960
961
966
967
968
969
976
979
981
982
984
985
996
998


In [26]:
i

6

In [27]:
infer_stats[18]['beams'][0]['inferred_code']

'SELECT some_col, some_col FROM stadium WHERE some_col = (SELECT Min(some_col) FROM stadium)'

In [52]:
dev = json.load(open('../data/spider-20190205/dev.json'))

In [71]:
num_total = 0
num_exact_match = 0
num_exact_or_exec_match = 0
num_errors = 0
num_errors_no_exact_match = 0

search_recovered_indices = []
search_failed_indices = []

sketch_search_results = {}
for line in open('../logdirs/20190315-sketch/upd_steps=4,max_steps=80000,batch_size=10/sketch-search-step68100-bs1.jsonl'):
    result = json.loads(line)
    num_total += 1
    sketch_exact_match = bool(best_stats['per_item'][result['index']]['exact'])
    num_exact_match += sketch_exact_match
    num_exact_or_exec_match += sketch_exact_match or bool(result['correct_sqls'])
    num_errors += bool(result['error'])
    num_errors_no_exact_match += not sketch_exact_match and bool(result['error'])
    
    if not sketch_exact_match:
        if bool(result['correct_sqls']):
            search_recovered_indices.append(result['index'])
        else:
            search_failed_indices.append(result['index'])

    sketch_search_results[result['index']] = result

In [72]:
num_total

1032

In [73]:
num_exact_match / num_total

0.6472868217054264

In [74]:
num_exact_or_exec_match / num_total

0.7238372093023255

In [75]:
num_errors / num_total

0.14631782945736435

In [76]:
num_errors_no_exact_match / num_total

0.07461240310077519

In [77]:
for i in search_recovered_indices[:10]:
    print('db_id: {}'.format(dev[i]['db_id']))
    print('Question: {}'.format(dev[i]['question']))
    print('Gold SQL: {}'.format(dev[i]['query']))
    print('Inferred sketch: {}'.format(sketch_search_results[i]['sketch_sql']))
    print('Correct sketch instantiations:')
    for correct in sketch_search_results[i]['correct_sqls'][:5]:
        print('  {}'.format(correct))
    print()

db_id: concert_singer
Question: Show the name and the release year of the song by the youngest singer.
Gold SQL: SELECT song_name ,  song_release_year FROM singer ORDER BY age LIMIT 1
Inferred sketch: SELECT some_col, some_col FROM stadium WHERE some_col = (SELECT Min(some_col) FROM stadium)
Correct sketch instantiations:
  SELECT singer.Song_Name, singer.Song_release_year FROM singer WHERE singer.Age = (SELECT Min(singer.Age) FROM singer)

db_id: concert_singer
Question: Show the stadium name and capacity with most number of concerts in year 2014 or after.
Gold SQL: SELECT T2.name ,  T2.capacity FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id WHERE T1.year  >=  2014 GROUP BY T2.stadium_id ORDER BY count(*) DESC LIMIT 1
Inferred sketch: SELECT some_col, some_col FROM stadium WHERE some_col > 'terminal' GROUP BY some_col
Correct sketch instantiations:
  SELECT stadium.Name, stadium.Capacity FROM stadium JOIN singer JOIN concert ON stadium.Stadium_ID = concert.St

In [62]:
print(open('../data/spider-20190205/database/flight_2/flight_2.sql').read())

CREATE TABLE 'airlines' (
	uid INTEGER PRIMARY KEY, 
	Airline TEXT, 
	Abbreviation TEXT, 
	Country TEXT
);
CREATE TABLE 'airports' (
	City TEXT, 
	AirportCode TEXT PRIMARY KEY, 
	AirportName TEXT, 
	Country TEXT, 
	CountryAbbrev TEXT
);
CREATE TABLE 'flights' (
	Airline INTEGER, 
	FlightNo INTEGER, 
	SourceAirport TEXT, 
	DestAirport TEXT, 
	PRIMARY KEY(Airline, FlightNo),
	FOREIGN KEY (SourceAirport) REFERENCES airports(AirportCode),
	FOREIGN KEY (DestAirport) REFERENCES airports(AirportCode)
);



In [79]:
for i in search_failed_indices[:10]:
    print('Question: {}'.format(dev[i]['question']))
    print('Gold SQL: {}'.format(dev[i]['query']))
    print('Inferred sketch: {}'.format(sketch_search_results[i]['sketch_sql']))
    print()

Question: What is the average, minimum, and maximum age for all French singers?
Gold SQL: SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'
Inferred sketch: SELECT Avg(some_col), Min(some_col), Max(some_col) FROM stadium

Question: What is the name and capacity for the stadium with highest average attendance?
Gold SQL: SELECT name ,  capacity FROM stadium ORDER BY average DESC LIMIT 1
Inferred sketch: SELECT some_col, some_col FROM stadium WHERE some_col = 'terminal' GROUP BY some_col ORDER BY Avg(some_col) Desc LIMIT 1

Question: What is the name and capacity for the stadium with the highest average attendance?
Gold SQL: SELECT name ,  capacity FROM stadium ORDER BY average DESC LIMIT 1
Inferred sketch: SELECT some_col, some_col FROM stadium WHERE some_col = 'terminal' GROUP BY some_col ORDER BY Avg(some_col) Desc LIMIT 1

Question: What is the name and capacity of the stadium  with the most concerts after 2013?
Gold SQL: SELECT T2.name ,  T2.capacity FROM

In [59]:
dev[0]

{'db_id': 'concert_singer',
 'query': 'SELECT count(*) FROM singer',
 'query_toks': ['SELECT', 'count', '(', '*', ')', 'FROM', 'singer'],
 'query_toks_no_value': ['select', 'count', '(', '*', ')', 'from', 'singer'],
 'question': 'How many singers do we have?',
 'question_toks': ['How', 'many', 'singers', 'do', 'we', 'have', '?'],
 'sql': {'except': None,
  'from': {'conds': [], 'table_units': [['table_unit', 1]]},
  'groupBy': [],
  'having': [],
  'intersect': None,
  'limit': None,
  'orderBy': [],
  'select': [False, [[3, [0, [0, 0, False], None]]]],
  'union': None,
  'where': []}}